<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemple d'apprentissage du ET avec TensorFlow
## Illustration de l'importance du Bias

|a|b|a et b|
|:-:|:-:|:-:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones n'est pas possible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `a` et `b`.
`wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(0)` tend vers 0 --> ok
 - `f(wb)` tend vers 0
 - `f(wa)` tend vers 0
 - `f(wa + wb)` tend vers 1 --> conflit avec les lignes précédentes
 
*Vérifions le...*


---
**Importer les librairies**

In [1]:
#keras : Python Deep Learning library
import tensorflow.keras as keras
#prevision d'utiliser un réseau en couches séquentielles
from tensorflow.keras.models import Sequential
#prevision d'utiliser des couches totalement connectées la précédente
from tensorflow.keras.layers import Dense
#utilisation de la classique librairie pour tableaux, ...
import numpy as np

---

## Définir les entrées et sorties attendues


In [2]:
# a et b sont les seules entrées
entrees = np.array([[0,0],[0,1],[1,0],[1,1]], float)

# une seule sortie
sorties = np.array([[0],[0],[0],[1]], float)

---
## 1. Version sans BIAS

### 1.1. Choisir le modèle de réseau 
***ici les couches sont séquentielles***

In [3]:
model = Sequential()

### 1.2. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 2 neurones en entrée (pour chaque valeur), 
- utilisation de la sigmoïde comme fonction d'activation

In [4]:
model.add(Dense(1, input_dim=2, use_bias=False, activation='sigmoid'))

---

### 1.3. Compiler le  réseau
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des valeurs absolues des erreurs commises

In [5]:
model.compile(optimizer='adamax', loss='MSE')

---

### 1.4. Entraîner le réseau 
- ici on ne le fait pas parler (verbose = 0), 
- et on lance 10000 cycles d'apprentissage

In [6]:
model.fit(entrees, sorties, verbose=0, epochs=10000)

---

### 1.5. Vérifier le réseau
Etape facultative, en général ***on teste le réseau sur d'autres exemples***. 
- Ici, on n'en a pas. Alors on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement

In [7]:
predictions = model.predict(entrees)

---
### 1.6. Affichage des résultats
Ici pas de nécessité de graphique d'évolution de l'erreur.
On affiche les entrées, la sortie attendue, la sortie calculée ainsi que les poids appliquées aux entrées et au signal bias..

In [8]:
def verification(bias=False):
    print("verification")
    for i in range(0, len(entrees)):
        print(entrees[i][0], " - ", entrees[i][1], " attendu ", sorties[i], " trouvé ",  predictions[i])

    ws = model.get_weights()
    print("poids pour entree x = " + str(ws[0][0][0]))
    print("poids pour entree y = " + str(ws[0][1][0]))
    if(bias):print("poids pour bias = " + str(ws[1][0]))

verification()

verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.5]
0.0  -  1.0  attendu  [0.]  trouvé  [0.5000002]
1.0  -  0.0  attendu  [0.]  trouvé  [0.50000006]
1.0  -  1.0  attendu  [1.]  trouvé  [0.50000024]
poids pour entree x = 2.6033268e-07
poids pour entree y = 6.0059847e-07


In [9]:
loss = model.evaluate(entrees, sorties,verbose=0)
print("perte=",loss)

perte= 0.25


**Des erreurs importantes donc**, comme prévu....

---
## 2. Version AVEC BIAS

Le tableau est alors

|bias|a|b|a et b|
|:-:|:-:|:-:|:-:|
|1|0|0|0|
|1|0|1|0|
|1|1|0|0|
|1|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones est alors possible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `bias`, `a` et `b`.
`wbias`, `wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(bias)` tend vers 0
 - `f(bias + wb)` tend vers 0
 - `f(bias + wa)` tend vers 0
 - `f(bias + wa + wb)` tend vers 1 
 
*Vérifions le...*

---

### 2.1. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 3 neurones en entrée (2 contenant les valeurs + **un Bias** (émettant toujours le signal 1)), 
- utilisation de la sigmoide comme fonction d'activation

In [10]:
model = Sequential()
model.add(Dense(1, input_dim=2, use_bias=True, activation='sigmoid'))

---

### 2.2 Compiler et entrainer le  réseau 
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des valeurs absolues des erreurs commises
  - 10000 cycles d'apprentissage demandés

In [11]:
model.compile(optimizer='adamax', loss='MSE')

model.fit(entrees, sorties, verbose=0, epochs=10000)

---

### 2.2. Vérifier le réseau
Pas d'exemples de validation, on vérifie simplement la correspondance entre sortie attendue et la sortie réelle.

In [12]:
predictions = model.predict(entrees)
verification(True)

verification
0.0  -  0.0  attendu  [0.]  trouvé  [4.3541193e-05]
0.0  -  1.0  attendu  [0.]  trouvé  [0.03397343]
1.0  -  0.0  attendu  [0.]  trouvé  [0.03396708]
1.0  -  1.0  attendu  [1.]  trouvé  [0.9660015]
poids pour entree x = 6.694462
poids pour entree y = 6.6946545
poids pour bias = -10.042267


In [13]:
loss = model.evaluate(entrees, sorties, verbose=0)
print("perte=",loss)

perte= 0.0008659637533128262


**Apprentissage quasi parfait !!!** 
- -> démonstration concrère de l'effet du `Bias` !!